In [1]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

import statsmodels.api as sm
from scipy import stats
stats.chisqprob = lambda chisq, df: stats.chi2.sf(chisq, df)




# **Load data**

In [2]:
data_train = pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv',index_col = 'Id')
data_test = pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv',index_col = 'Id')


data_train.head()


,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [3]:
#Set up Screen output
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', 100)

In [4]:
data_train.describe()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1379.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,46.549315,567.240411,1057.429452,1162.626712,346.992466,5.844521,1515.463699,0.425342,0.057534,1.565068,0.382877,2.866438,1.046575,6.517808,0.613014,1978.506164,1.767123,472.980137,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,161.319273,441.866955,438.705324,386.587738,436.528436,48.623081,525.480383,0.518911,0.238753,0.550916,0.502885,0.815778,0.220338,1.625393,0.644666,24.689725,0.747315,213.804841,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,0.000000,0.000000,0.000000,334.000000,0.000000,0.000000,334.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,1900.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,0.000000,223.000000,795.750000,882.000000,0.000000,0.000000,1129.500000,0.000000,0.000000,1.000000,0.000000,2.000000,1.000000,5.000000,0.000000,1961.000000,1.000000,334.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,0.000000,477.500000,991.500000,1087.000000,0.000000,0.000000,1464.000000,0.000000,0.000000,2.000000,0.000000,3.000000,1.000000,6.000000,1.000000,1980.000000,2.000000,480.000000,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,0.000000,808.000000,1298.250000,1391.250000,728.000000,0.000000,1776.750000,1.000000,0.000000,2.000000,1.000000,3.000000,1.000000,7.000000,1.000000,2002.000000,2.000000,576.000000,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,1474.000000,2336.000000,6110.000000,4692.000000,2065.000000,572.000000,5642.000000,3.000000,2.000000,3.000000,2.000000,8.000000,3.000000,14.000000,3.000000,2010.000000,4.000000,1418.000000,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


**Check NaN values**

In [5]:
data_train.isnull().sum()


MSSubClass          0
MSZoning            0
LotFrontage       259
LotArea             0
Street              0
Alley            1369
LotShape            0
LandContour         0
Utilities           0
LotConfig           0
LandSlope           0
Neighborhood        0
Condition1          0
Condition2          0
BldgType            0
HouseStyle          0
OverallQual         0
OverallCond         0
YearBuilt           0
YearRemodAdd        0
RoofStyle           0
RoofMatl            0
Exterior1st         0
Exterior2nd         0
MasVnrType          8
MasVnrArea          8
ExterQual           0
ExterCond           0
Foundation          0
BsmtQual           37
BsmtCond           37
BsmtExposure       38
BsmtFinType1       37
BsmtFinSF1          0
BsmtFinType2       38
BsmtFinSF2          0
BsmtUnfSF           0
TotalBsmtSF         0
Heating             0
HeatingQC           0
CentralAir          0
Electrical          1
1stFlrSF            0
2ndFlrSF            0
LowQualFinSF        0
GrLivArea 

# Declare variables

In this session, I will spilt numerical columns and catagorical columns then I will use SimpleImputer to clean a data.

In [6]:
from sklearn.model_selection import train_test_split

#Declare the variable
X = data_train.drop(['SalePrice'], axis=1)
y = data_train['SalePrice']

#Split Train Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [cname for cname in X_train.columns if
                    X_train[cname].nunique() < 10 and 
                    X_train[cname].dtype == "object"]
# Select numerical columns
numerical_cols = [cname for cname in X_train.columns if 
                X_train[cname].dtype in ['int64', 'float64']]

all_cols = numerical_cols + categorical_cols
X_train = X_train[all_cols]
X_test = X_test[all_cols]
data_test = data_test[all_cols]

In [7]:
print(X_train.shape,X_test.shape,data_test.shape)

(1095, 76) (365, 76) (1459, 76)


# Preprocessing

In [8]:
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Condition1,Condition2,BldgType,HouseStyle,RoofStyle,RoofMatl,MasVnrType,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Heating,HeatingQC,CentralAir,Electrical,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1293,70,60.0,6600,5,4,1892,1965,0.0,0,0,994,994,1378,994,0,2372,0,0,2,0,4,2,11,0,1985.0,1,432,0,287,0,0,0,0,0,12,2009,RM,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,Norm,Norm,1Fam,2Story,Gable,CompShg,None,TA,TA,Stone,TA,TA,No,Unf,Unf,GasA,TA,N,SBrkr,TA,Min2,NaN,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
1019,80,NaN,10784,7,5,1991,1992,76.0,0,0,384,384,802,670,0,1472,0,0,2,1,3,1,7,1,1991.0,2,402,164,0,0,0,0,0,0,5,2007,RL,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,Norm,Norm,1Fam,SLvl,Gable,CompShg,BrkFace,Gd,TA,PConc,Gd,TA,No,Unf,Unf,GasA,Gd,Y,SBrkr,Gd,Typ,TA,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
1214,80,NaN,10246,4,9,1965,2001,0.0,648,0,0,648,960,0,0,960,1,1,0,0,0,1,3,0,1965.0,1,364,88,0,0,0,0,0,0,5,2006,RL,Pave,NaN,IR1,Lvl,AllPub,CulDSac,Gtl,Norm,Norm,1Fam,SLvl,Gable,CompShg,None,TA,Gd,CBlock,TA,Gd,Av,GLQ,Unf,GasA,Ex,Y,SBrkr,TA,Typ,NaN,Attchd,Unf,TA,TA,Y,NaN,NaN,NaN,WD,Normal
1431,60,60.0,21930,5,5,2005,2005,0.0,0,0,732,732,734,1104,0,1838,0,0,2,1,4,1,7,1,2005.0,2,372,100,40,0,0,0,0,0,7,2006,RL,Pave,NaN,IR3,Lvl,AllPub,Inside,Gtl,RRAn,Norm,1Fam,2Story,Gable,CompShg,None,Gd,TA,PConc,Gd,Gd,Av,Unf,Unf,GasA,Ex,Y,SBrkr,TA,Typ,Gd,BuiltIn,Fin,TA,TA,Y,NaN,NaN,NaN,WD,Normal
811,20,78.0,10140,6,6,1974,1999,99.0,663,377,0,1040,1309,0,0,1309,1,0,1,1,3,1,5,1,1974.0,2,484,265,0,0,0,0,648,0,1,2006,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Norm,Norm,1Fam,1Story,Hip,CompShg,BrkFace,TA,TA,CBlock,TA,TA,No,ALQ,LwQ,GasA,Fa,Y,SBrkr,Gd,Typ,Fa,Attchd,RFn,TA,TA,Y,Fa,GdPrv,NaN,WD,Normal


In [9]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder

num_imputer = SimpleImputer(strategy='median')
cat_imputer = SimpleImputer(strategy='most_frequent')

# Preprocessing for numerical data
for cname in X_train.columns:
    if X_train[cname].dtype in ['int64', 'float64'] :
        X_train[cname] = num_imputer.fit_transform(X_train[[cname]])

for cname in X_test.columns:
    if X_test[cname].dtype in ['int64', 'float64'] :
        X_test[cname] = num_imputer.fit_transform(X_test[[cname]])

for cname in data_test.columns:
    if data_test[cname].dtype in ['int64', 'float64']:
        data_test[cname] = num_imputer.fit_transform(data_test[[cname]])

# Preprocessing for categorical data

for cname in categorical_cols:
        X_train[cname] = cat_imputer.fit_transform(X_train[[cname]])
        
for cname in categorical_cols:
    if X_test[cname].nunique() < 10 and X_test[cname].dtype == "object":
        X_test[cname] = cat_imputer.fit_transform(X_test[[cname]])

for cname in categorical_cols:
    if data_test[cname].nunique() < 10 and data_test[cname].dtype == "object":
        data_test[cname] = cat_imputer.fit_transform(data_test[[cname]])

# Apply label encoder
label_encoder = LabelEncoder()

for cname in categorical_cols:
    X_train[cname] = label_encoder.fit_transform(X_train[cname])
    X_test[cname] = label_encoder.fit_transform(X_test[cname])
    data_test[cname] = label_encoder.fit_transform(data_test[cname])


In [10]:
print(X_train.shape,X_test.shape,data_test.shape)

(1095, 76) (365, 76) (1459, 76)


In [11]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

model = XGBRegressor(n_estimators = 500, learning_rate = 0.01)
model.fit(X_train, y_train,early_stopping_rounds=5, 
               eval_set=[(X_test, y_test)],
               verbose=False )

y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

print('Train MAE :', mean_absolute_error(y_train,y_train_pred))
print('Test MAE :', mean_absolute_error(y_test,y_test_pred))
print('Train MSE :', mean_squared_error(y_train,y_train_pred))
print('Test MSE :', mean_squared_error(y_test,y_test_pred))
print('RMSE Train :', np.sqrt(mean_squared_error(y_train,y_train_pred)))
print('RMSE Test :', np.sqrt(mean_squared_error(y_test,y_test_pred)))
print('R2 Train :', r2_score(y_train,y_train_pred))
print('R2 Test :', r2_score(y_test,y_test_pred))

Train MAE : 6317.277051227169
Test MAE : 16779.879965753425
Train MSE : 73482773.80574983
Test MSE : 1051114859.673884
RMSE Train : 8572.209388818605
RMSE Test : 32420.901586382264
R2 Train : 0.9881637298110898
R2 Test : 0.8407864120554299


/opt/conda/lib/python3.7/site-packages/xgboost/data.py:114: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "because it will generate extra copies and increase " +


In [12]:
predictions = model.predict(data_test)
data_for_sub = pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv')
ID = data_for_sub['Id']
submission  = pd.DataFrame({'ID': ID,'SalePrice':predictions})
submission.to_csv('submission.csv',index=False)

/opt/conda/lib/python3.7/site-packages/xgboost/data.py:114: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "because it will generate extra copies and increase " +
